# Weekly Assignment 3: Part 2 

Imported the necessary libraries

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In the following code, I have accessed the table data from the url and formed a dataframe with the 3 headings as required. Futhermore, i have filtered out row 0 which did not have any data using the isnull function for Postal Code

In [2]:
import requests
from bs4 import BeautifulSoup
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df.shape

(180, 3)

As we can see form the above table, there are several 'Not assigned' values under Borough. To drop these rows, I have used the following code and accordingly reindexed the data. For this data, all assigned Boroughs have assigned Neighbourhood details, hence we do not need to drop not assigned Neighbourhood rows. Furthermore, there are no repeating postal codes.

In [9]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df1 = df.reset_index()
df1.head(15)

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,10,M1B,Scarborough,"Malvern, Rouge"
7,12,M3B,North York,Don Mills
8,13,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,14,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df3= df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df3.shape

(103, 3)

Since I faced several issues regarding the Geocoder package, I have used the coordinates provided by the csv file in the instructions. I viewed the data in excel and found that on doing an inner join, the Postal Code column data was repeating due to difference in variable name. Hence, i dropped the Postal Code Column from the second dataset to ensure there was no repetition. 

In [7]:
df4 = pd.read_csv ('https://cocl.us/Geospatial_data')
merged_inner = pd.merge(left=df3, right= df4, left_on='PostalCode', right_on='Postal Code')
Data = merged_inner.drop(columns="Postal Code")
Data.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB
